The requirement is that with just 3D meshes, time difference information is lost. We need to keep the time difference in the data as much as possible.

# XYZ Points

Here we want to try if pointnet can understand just `x, y, z` points. `time` is not part of this data. 

NOTE: Need to evenly sample points for each timeslice.

Techniques:
<uo>
    <li> Change Class Weights in PyTorch to make it pay more attention to undersampled classes
    <li> Upsample 
    <li> Downsample
    <li> Metrics
        <uo>
            <li> Area under ROC curve
        </uo>
</uo>

**Q:** How unevenly is the data imbalanced? Hits vs noise points rate.

#### Exp 1.0:

**Date:** 17-July-2020

**Rationale:**

**Parameters:**

**Train Results:**

**Test Results:**

**Comments:**

# 3D Meshes

#### Exp 1.0:

**Date:** 09-Jul-2020

**Parameters:**
1024 points
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. logsoftmax

**Train Results:**
50 %, 40 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %

**Test Results:**
<img src="../assets/experiments/exp_1.0.png">

#### Exp 2.0:

**Date:** 10-Jul-2020

**Parameters:**
1024 points
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. **Changed: LogSigmoid** 

**Train Results:**
50 %, 40 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 54 %, 50 %, 59 %

**Test Results:**
 <img src="../assets/experiments/exp_2.0.png">

##### Exp 2.1:

**Date:** 10-Jul-2020

**Parameters:**
1024 points
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. **Changed: Sigmoid** 

**Rationale**

Does changing between log sigmoid vs sigmoid have any effect on results?
It actually decreased training performance

**Train Results:**
50 %, 40 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 45 %, 45 %

**Test Results:**
 <img src="../assets/experiments/exp_2.1.png">

#### Exp 3.0: 
**Date:** -Jul-2020

**Results:**
1024 Max points
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. LogSigmoid
5. **Changed:LossFunction - BinaryEntropyLoss**

**Rationale**
The loss function for predicting binary outcomes should be Binary Loss Entropy

**Result**

No real difference between negative log likelihood loss mathematically

##### Exp 3.1:
**Date:** -Jul-2020

**Parameters:**
1024 Max points
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. LogSigmoid
5. **Changed:LossFunction - CrossEntropyLoss**

**Rationale**
Binary Loss Entropy did not work. Tried Crossentropy (multiclass as binary problem)

**Train Results:**
50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 54 %, 54 %

**Test Results:**
<img src="../assets/experiments/exp_3.1.png">

#### Exp 4.0:
**Date:** 13-Jul-2020

**Parameters: Evaluating a Larger Model**
**Changed: Max points to 2048**
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**
50 %, 40 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %, 50 %

**Remarks**
No real difference

**Test Results:**
<!-- <img src="../assets/experiments/"> -->

#### Exp 5.0:
**Date:** 13-Jul-2020

**Parameters:**
**Changed: Increased files to 200 per type**
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**
50 %, 50 %, 50 %, 55 %, 43 %, 45 %, 53 %, 60 %, 50 %, 50 %, 47 %, 47 %, 50 %, 47 %, 46 %


[Epoch: 1, Batch:   10 /   10], loss: 0.569

[Epoch: 2, Batch:   10 /   10], loss: 0.243

[Epoch: 3, Batch:   10 /   10], loss: 0.126

[Epoch: 4, Batch:   10 /   10], loss: 0.074

[Epoch: 5, Batch:   10 /   10], loss: 0.050

[Epoch: 6, Batch:   10 /   10], loss: 0.038

[Epoch: 7, Batch:   10 /   10], loss: 0.028

[Epoch: 8, Batch:   10 /   10], loss: 0.022

[Epoch: 9, Batch:   10 /   10], loss: 0.019

[Epoch: 10, Batch:   10 /   10], loss: 0.016

[Epoch: 11, Batch:   10 /   10], loss: 0.014

[Epoch: 12, Batch:   10 /   10], loss: 0.013

[Epoch: 13, Batch:   10 /   10], loss: 0.011

[Epoch: 14, Batch:   10 /   10], loss: 0.010

[Epoch: 15, Batch:   10 /   10], loss: 0.009


**Remarks**
Produced 60% validation scores. Loss decrease rate stabalises around 10-11 epochs

**Test Results:**
<img src="../assets/experiments/exp_5.0.png">

##### Exp 5.1:
**Date:** 13-Jul-2020

**Parameters:**
200 Files/Class
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. **Changed: Sigmoid **
5. NNLoss

**Train Results:**
50 %, 50 %, 50 %, 55 %, 43 %, 45 %, 53 %, 60 %, 50 %, 50 %, 47 %, 47 %, 50 %, 47 %, 46 %


**Rationale**
Sigmoid produced a more precise confusion matrix despite lower scores


**Remarks**
Produced 60% validation scores. Loss decrease rate stabalises around 10-11 epochs

**Test Results:**
<img src="../assets/experiments/exp_5.0.png">

# 4D Mesh

# LSTM Units

# (XYT, XZT, YZT) + Ensemble

Still to try:
1. Oversample 
2. Undersample
3. Larger Dataset
4. Non Sampled dataset

#### Exp 1.0: 
**Date:** 05-August-2020

**Condition:** 6550 points per timeslice were taken. Hits ordered first so
that they would be selected first and the balance would be noise points

**Parameters:**
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**
[Epoch: 14, Batch:   10 /   10], loss: 0.006
Valid accuracy: 97 %

**Classification report**

               precision    recall  f1-score   support

       mixed       1.00      0.93      0.96        40
       noise       0.93      1.00      0.96        40

    accuracy                           0.96        80
    macro avg       0.97      0.96     0.96        80
    weighted avg    0.97      0.96     0.96        80



<img src="../assets/experiments/ensemble/exp_1.0.png">

#### Exp 1.1: 
**Date:** 05-August-2020

**Condition**

Test data must not be 6550 points (Previously, train data )

1. Split to train/test
2. Train is resampled down to 6550 pints only
3. Test is left untouched


**Parameters:**
1. Sampled, normalised, 6550 points
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**
[Epoch: 14, Batch:   10 /   10], loss: 0.009
Valid accuracy: 50 %

**Classification report**

Classification report for Pointnet:

               precision    recall  f1-score   support

       mixed       0.29      0.10      0.15        40
       noise       0.45      0.75      0.57        40

    accuracy                           0.42        80
       macro avg       0.37      0.42      0.36        80
    weighted avg       0.37      0.42      0.36        80

<img src="../assets/experiments/ensemble/exp_1.1.png">

#### Exp 1.2: 
**Date:** 08-August-2020

**Condition**
6550 train, all test


1. Split to train/test
2. Train is resampled down to 6550 pints only
3. Test is left untouched

**Parameters:**
1. Sampled, normalised, 1024 points 
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**
[Epoch: 10, Batch:   10 /   10], loss: 0.010
Valid accuracy: 61 %


**Classification report**


Classification report for Pointnet:

               precision    recall  f1-score   support

       mixed       0.60      0.23      0.33        40
       noise       0.52      0.85      0.65        40

    accuracy                           0.54        80
   macro avg       0.56      0.54      0.49        80
weighted avg       0.56      0.54      0.49        80

<img src="../assets/experiments/ensemble/exp_1.1.png">

#### Exp 1.3: 
**Date:** 08-August-2020

**Condition**
6550 train, all test


1. Split to train/test
2. Train is resampled down to 6550 pints only
3. Test is left untouched

**Parameters:**
1. Sampled, normalised, 1024 points 
2. Rotated
3. Added Noise
4. **Sigmoid**
5. **BCELoss**
5. **30 epochs**

**Train Results:**
Valid accuracy: 55 %
[Epoch: 23, Batch:   10 /   10], loss: 0.114

Also, loss was fluctuating and never settled into minima

**Classification report**


Classification report for Pointnet:

               precision    recall  f1-score   support

       mixed       0.52      0.60      0.56        40
       noise       0.53      0.45      0.49        40

    accuracy                           0.53        80
    macro avg       0.53      0.53     0.52        80
    weighted avg    0.53      0.53     0.52        80

<img src="../assets/experiments/ensemble/exp_1.3.png">

#### Exp 1.4: 
**Date:** 08-August-2020

**Condition**
6550 train, all test


1. Split to train/test
2. Train is resampled down to 6550 pints only
3. Test is left untouched

**Parameters:**
1. Sampled, normalised, 1024 points 
2. Rotated
3. Added Noise
4. **BCEWithLogitsLoss**
5. **60 epochs**

**Train Results:**
[Epoch: 12, Batch:   10 /   10], loss: 0.130
Valid accuracy: 65 %

Also, loss was fluctuating and never settled into minima

**Classification report**

Classification report for Pointnet:

               precision    recall  f1-score   support

       mixed       0.52      0.35      0.42        40
       noise       0.51      0.68      0.58        40

    accuracy                           0.51        80
    macro avg       0.51      0.51     0.50        80
    weighted avg    0.51      0.51     0.50        80

<img src="../assets/experiments/ensemble/exp_1.4.png">

#### Exp 1.5: 
**Date:** 08-August-2020

**Condition**
6550 train, all test


1. Split to train/test
2. Train is resampled down to 6550 pints only
3. Test is left untouched

**Parameters:**
1. Sampled, normalised, 1024 points 
2. Rotated
3. Added Noise
4. **Cross Entropy Loss (includes sigmoid)**
5. **80 epochs**

**Train Results:**
[Epoch: 59, Batch:   10 /   10], loss: 0.708
Valid accuracy: 65 %

Also, loss was fluctuating and never settled into minima (0.710-0.687)

**Classification report**

Classification report for Pointnet:

               precision    recall  f1-score   support

        mixed       0.80      0.40      0.53        40
        noise       0.60      0.90      0.72        40

     accuracy                           0.65        80
    macro avg       0.70      0.65      0.63        80
    weighted avg    0.70      0.65      0.63        80

<img src="../assets/experiments/ensemble/exp_1.5.png">

#### Exp 2.0: 
**Date:** 05-August-2020

**Condition:** No points sampled

**Parameters:**
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**
[Epoch: 14, Batch:   10 /   10], loss: 0.009
Valid accuracy: 57 %

**Classification report**

               Classification report for Pointnet:

               precision    recall  f1-score   support

       mixed       0.38      0.15      0.21        40
       noise       0.47      0.75      0.58        40

    accuracy                           0.45        80
    macro avg       0.42      0.45     0.40        80
    weighted avg    0.42      0.45     0.40        80




<img src="../assets/experiments/ensemble/exp_2.0.png">

#### Exp 2.1: 
**Date:** 08-August-2020

**Condition:** No points sampled
    

**Parameters:**
1. Sampled, normalised, 1024 points (instead of 6550)
2. Rotated
3. Added Noise
4. LogSigmoid
5. NNLoss

**Train Results:**


**Classification report**

        



<img src="../assets/experiments/ensemble/exp_2.1.png">

#### Exp 3.0: 
**Date:** 08-August-2020

**Condition:** Classes set to **k = 1** (instead of k = 2)

**Parameters:**
1. Sampled, normalised,
2. Rotated
3. Added Noise
4. Sigmoid
5. BCE
6. 80 epochs

**Train Results:**


**Classification report**

            



<img src="../assets/experiments/ensemble/exp_3.0.png">

#### Exp 4.0: Ensemble Best Loss Functiom

Choosing the best loss function: BCEWith Logits, NLLLoss, CrossEntropyLoss

**Date:** 15-August-2020

No 6550 points, full data

**Condition:** 
Batch:[32, 64]
Epoch: 250    
    
**Parameters:**
1. Sampled, normalised,
2. Rotated
3. Added Noise


                   BCEWLogitsLoss     NLLoss      CELoss
       Recall
       
       mixed           0.62             0.53       0.35  
       noise           0.40             0.55       0.55


       FPR             0.65             0.475       0.60
       
   
BCEWithLogitsLoss gives highest acccuraccy but NLLoss gives lowest FPR overall.

**Result: Proceed with NLLoss.

# Remarks

The output neuron looked something like this: [32, 2] matrix indicating that one neuron was assigned for. each class (k = 2). This makes it NOT a binary classification problem but a **Predicting a single label from multiple classes** problem.

## Predicting a single label from multiple classes ##
The final layer of the neural network will have one neuron for each of the classes and they will return a value between 0 and 1, which can be inferred as a probably. The output then results in a probability distribution as it sums to 1.
To understand the accuracy of the prediction, each output is compared with its corresponding true value. True values have been one-hot-encoded meaning a 1 appears in the column corresponding to the correct category, else a 0 appears


Softmax with cross entropy should be tried


## Binary Classification Loss Functions [UPDATE: INVALID]

[o] ***Binary Cross-Entropy:***
Cross-entropy is the default loss function to use for binary classification problems. 
It is intended for use with binary classification where the target values are in the set {0, 1}. Mathematically, it is the preferred loss function under the inference framework of maximum 
likelihood. 
It is the loss function to be evaluated first and only changed if you have a good reason.
Cross-entropy will calculate a score that summarizes the average difference between the actual and predicted probability distributions for predicting class 1. 
The score is minimized and a perfect cross-entropy value is 0.


[x] ***Hinge Loss:*** 
An alternative to cross-entropy for binary classification problems is the hinge loss function, primarily developed for use with Support Vector Machine (SVM) models.It is intended for use with binary classification where the target values are in the set {-1, 1}.

[x] ***Squared Hinge Loss:*** 
Calculates the square of the score hinge loss. It has the effect of smoothing the surface of the error function and making it numerically easier to work with. If using a hinge loss does result in better performance on a given binary classification problem, is likely that a squared hinge loss may be appropriate. As with using the hinge loss function, the target variable must be modified to have values in the set {-1, 1}.

# Test Available GPU on Colab
<a id='gpu-test'></a>

```python
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]

def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() ```

In [1]:
def train(model, train_loader, val_loader=None,  epochs=15, save=True):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:
                print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' % 
                      (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                running_loss = 0.0

        pointnet.eval()
        correct = total = 0
    

Noted that the read method converts to floats that looses several points

Adding a 4th dimension: https://github.com/charlesq34/pointnet/issues/213
https://github.com/charlesq34/pointnet/issues/12
https://github.com/fxia22/pointnet.pytorch/issues/69


Helpful visualisations: [https://github.com/Wind-Wing/pointnet-visualization/blob/master/visualization_report.pdf]